In [1]:
STOPWORDS = {
    "the","and","a","to","of","in","that","it","is","for","on","as",
    "with","was","be","by","he","she","you","i","are","at","from"
}

def apply_stopwords(tok_list):
    return [t for t in tok_list if t not in STOPWORDS]